# CS 182: Mike

We decided to use two data sets.

# Github Data 

Our primary data set is from Peter Downs via Github.

Link: https://github.com/peterldowns/clickbait-classifier

In [182]:
%matplotlib inline
import datetime
import json
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import re
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style('darkgrid')
sns.set_context('poster')
import warnings
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)
warnings.simplefilter(action = 'ignore', category = DeprecationWarning)

In [183]:
# collect data
df1 = pd.read_json('https://raw.githubusercontent.com/peterldowns/clickbait-classifier/master/data/buzzfeed.json')
df2 = pd.read_json('https://raw.githubusercontent.com/peterldowns/clickbait-classifier/master/data/clickhole.json')
df3 = pd.read_json('https://raw.githubusercontent.com/peterldowns/clickbait-classifier/master/data/dose.json')
df4 = pd.read_json('https://raw.githubusercontent.com/peterldowns/clickbait-classifier/master/data/nytimes.json')

In [184]:
print 'There are', len(df1), 'Buzzfeed,', len(df4), 'NYT,', len(df2), 'Clickhole, and', len(df3), 'Daily Dose articles in the Github data set.'

There are 367 Buzzfeed, 3104 NYT, 547 Clickhole, and 2182 Daily Dose articles in the Github data set.


In [185]:
# label data before concatenating
df1['source'] = 'Buzzfeed'
df2['source'] = 'Clickhole'
df3['source'] = 'Daily Dose'
df4['source'] = 'NY Times'

In [186]:
# join publications
gh_data = pd.concat([df1[['article_title','article_url','clickbait','source']], df2, df3])
print gh_data.head()

                                       article_title                                        article_url  clickbait    source
0             23 Life Lessons Cosmo Kramer Taught Us  /javiermoreno/ife-lessons-you-learned-from-cos...          1  Buzzfeed
1          32 Men On TV Who Made You Thirsty In 2014  /erinlarosa/32-tv-men-who-made-you-thirsty-in-...          1  Buzzfeed
2          Hilary Duff Was The Walking Queen Of 2014  /lyapalater/hilary-duff-was-the-walking-queen-...          1  Buzzfeed
3        25 Reasons Wine Is Definitely Your Soulmate  /emleschh/25-reasons-why-wine-is-your-soulmate...          1  Buzzfeed
4  This Master Carver Making Pliers From One Stic...          /norbertobriceno/ernest-macguyver-warther          1  Buzzfeed


In [23]:
# join publications
gh_data = pd.concat([df1[['article_title','article_url','clickbait','source']], df2, df3, df4[['article_title','clickbait','source']]])
print gh_data.head()

                                       article_title                                        article_url  clickbait    source
0             23 Life Lessons Cosmo Kramer Taught Us  /javiermoreno/ife-lessons-you-learned-from-cos...          1  Buzzfeed
1          32 Men On TV Who Made You Thirsty In 2014  /erinlarosa/32-tv-men-who-made-you-thirsty-in-...          1  Buzzfeed
2          Hilary Duff Was The Walking Queen Of 2014  /lyapalater/hilary-duff-was-the-walking-queen-...          1  Buzzfeed
3        25 Reasons Wine Is Definitely Your Soulmate  /emleschh/25-reasons-why-wine-is-your-soulmate...          1  Buzzfeed
4  This Master Carver Making Pliers From One Stic...          /norbertobriceno/ernest-macguyver-warther          1  Buzzfeed


In [187]:
def removeNonAscii(s): return "".join(i for i in s if ord(i)<128)

def removeQuotations(s):
    counter = 0
    for i in s:
        if i == '\"':
            counter += 1
    return not (counter == 2 or counter == 0)

def removeParens(s):
    counter = 0
    for i in s:
        if i == '(' or i == ')':
            counter += 1
    return not (counter == 2 or counter == 0)

titles = open('article_titles.txt', 'w')
counter = 0
for i in gh_data['article_title']:
    nonAscii = removeNonAscii(i)
    strlist = list(nonAscii)
    lastchar = strlist[len(strlist) - 1]
    if lastchar != '.' and lastchar != '!' and lastchar != '?':
        strlist.append('.')
    nonAscii = ''.join(strlist)
    tokens = nonAscii.split()
    if len(tokens) < 4:
        continue
    for token in tokens:
        if removeQuotations(token):
            token = token.replace('\"', "")
        if removeParens(token):
            token = token.replace('(', "")
            token = token.replace(')', "")
        if token.isupper():
            if len(token) != 1 or token[0] == 'A':
                token = token.lower()
                tokenlist = list(token)
                tokenlist[0] = tokenlist[0].upper()
                token = "".join(tokenlist)
        tokenlist = list(token)
        tokenlist.append('\n')
        titles.write(''.join(tokenlist))
titles.close()

In [188]:
words = open('article_titles.txt', 'r')

wordMap = {}
extantTitles = []
currentTitle = []
totalWords = 0
for line in words:
    # Construct mapping of words to ints
    line = line.strip()
    if line == '':
        continue
    if line not in wordMap:
        wordMap[line] = len(wordMap)
    
    # Construct set of article titles to test for similarity
    currentTitle.append(wordMap[line])
    lastchar = line[len(line)-1]
    if (lastchar == '.' or lastchar == '?' or lastchar == '!'):
        extantTitles.append(currentTitle)
        totalWords += len(currentTitle)
        currentTitle = []
    
words.close()
print totalWords
print len(extantTitles)

37769
4100


In [238]:
words = open('article_titles.txt', 'r')
transitionMatrix = {}
wordFrequency = {}
beginnerFrequency = {}

prevToken = None
nextWordStart = True
for line in words:
    line = line.strip()
    if line == '':
        continue
    
    if line not in wordFrequency:
        wordFrequency[line] = 0
    wordFrequency[line] += 1
    
    if nextWordStart:
        if line not in beginnerFrequency:
            beginnerFrequency[line] = 0
        beginnerFrequency[line] += 1
        nextWordStart = False
    
    if prevToken == None:
        prevToken = line
    else:
        if prevToken not in transitionMatrix:
            transitionMatrix[prevToken] = {}
        if line not in transitionMatrix[prevToken]:
            transitionMatrix[prevToken][line] = 0
        transitionMatrix[prevToken][line] += 1
    
    lastchar = line[len(line)-1]
    if (lastchar == '.' or lastchar == '?' or lastchar == '!'):
        prevToken = None
        nextWordStart = True
    else:
        prevToken = line

In [239]:
print transitionMatrix['Cook']

{'This': 1, 'But': 1}


In [243]:
def constructDist(keys):
    frequencies = []
    for key in keys:
        frequencies.append(wordFrequency[key])
    return map(lambda x: x / float(sum(frequencies)), frequencies)
        
def constructLocalDist(keys, frequencyTable):
    frequencies = []
    for key in keys:
        frequencies.append(frequencyTable[key])
    return map(lambda x: x / float(sum(frequencies)), frequencies)

# Need to place more weight on more common words
import random

restrictStarters = False

phrase = []
prevWord = None
counter = 0
while True:
    while True:
        if prevWord == None:
            if restrictStarters:
                keys = transitionMatrix.keys()
                word = np.random.choice(keys, 1, constructDist(keys))
                phrase.append(word[0])
                prevWord = word[0]
            else:
                keys = beginnerFrequency.keys()
                word = np.random.choice(keys, 1, constructLocalDist(keys, beginnerFrequency))
                phrase.append(word[0])
                prevWord = word[0]
        else:
            if prevWord not in transitionMatrix:
                phrase = []
                prevWord = None
            else:
                keys = transitionMatrix[prevWord].keys()
                word = np.random.choice(keys, 1, constructLocalDist(keys, transitionMatrix[prevWord]))
                prevWord = word[0]
                phrase.append(word[0])
                lastchar = word[0][len(word[0])-1]
                if (lastchar == '.' or lastchar == '?' or lastchar == '!'):
                    break
    if len(phrase) < 5:
        phrase = []
        prevWord = None
    else:
        counter += 1
        print " ".join(phrase)
        phrase = []
        prevWord = None
    if counter > 5:
        break

Ultimate Drink Ideas In Harry Remains The Conditions Are Just Quit Their Love.
Literally To 'Rated R' Really Dumb...#5 Freaked Me Forever Shrouded In Again.
We'll Never Leave Them Was 12am Somewhere.
Diva Picture Of Harrison Fords Ankle In Memoriam: Remembering All Time!
Genius Responses From A Thousand Words.
Literally, These 24 Horrifying Old They Mean You Out...Yet You'll Understand.


In [85]:
phrase = []
prevWord = None
while True:
    if prevWord == None:
        word = random.choice(transitionMatrix.keys())
        phrase.append(word)
        print word
    else:
        if word not in transitionMatrix:
            phrase = []
            prevWord = None
        else:
            word = random.choice(transitionMatrix[prevWord].keys())
            phrase.append(word)
            lastchar = word[len(word)-1]
            if (lastchar == '.' or lastchar == '?' or lastchar == '!'):
                break
            print word
print word

Backgrounds,
Turkey
Slayer
Process
Gecko
but
Mending
Else
Task,
Slide,
Sweaters
whole
Troubled
Perfect
Irish
Transgender
Rooftop
blockbusters
Violent
Hand
Fly
Escalated
Other,
Amiably
Pursued
Survey
Zombie
Intends
Harry
6
Marathon:
2-0
Masturbates
Attack
Raised
Deaths
Bulls
Peres
Ya
Billy
Resist
Stuff
Pocahontas
Policy
1996
Vice-presidential
However
Thug,
Clueless
Inappropriate,
Hits,
Search
Unrest
Toyota
Habits
Simpsons...I
Motive
Flags
Sasquatch
Companies
Wait,
Glimpse,
Utilities
Reach
Out...#6
Landing,
7,000
Investors
French
Integration
Woo
Capitalize
Surgeries
Obligated
Students
Blunt
Bryan
Advise
Vacation,
Identifier
Newest,
Hanukkah
Windowless...And
End,
Abbas
Missiles,
Love,
Strike:
Possibly
Dwindle
Withdrawal
Rocky
Galls
Cubans
Remarkable
Jerusalem;
Bypass
Doubt
Angeles
Meet
Riven
Desert,
Homemade
Buried
Zac
Bosnia
Comb
Covert
Principals
Ewing
Survives
Sepulcher,
Tattered
Crossroads,
Brooklyn
Exclusive:
Their
Timing
Surges
Reinvents
Fiance
Manliest
Jiang's
Picture-Perfect
Has
E

ValueError: I/O operation on closed file

In [ ]:
words = open('article_titles.txt', 'r')
for line in words:
    for i in line:
        uniqueChars.add(i)
print uniqueChars
words.close()

In [13]:
import markovify

with open("article_titles.txt") as f:
    text = f.read()

# state_size=3 results look cleaner, but are less novel and more likely to closely resemble a headline in the corpus.
text_model = markovify.Text(text, state_size=2)

for i in range(5):
    print(text_model.make_sentence())

Matt Damon Was Nominated To Do With Your Haters.
The 1997 Elections: The Governor; Whitman Pleased at Victory In Vote Clinton Sought to Avoid, House Backs Ban On Human Cloning For Stem Cells.
This Artist Made, You Will Drool Over.
People Post The Craziest Things On Top Of Speakers And Play Loud Music, Magic Happens . This Girl Could Not Have Everyone Wanting Some Tea.
Sunburns That Hurt To Look Like Normal Disney Vacation Photos.


In [ ]:
    strlist = list(nonAscii)
    for idx, character in enumerate(strlist):
        #if character.isupper() and idx != 0 and strlist[idx - 1] != ' ' and strlist[idx - 1] != '\'"':
            #strlist[idx] = character.lower()
        if idx == len(strlist) - 1:
            if character != '.' and character != '!' and character != '?':
                strlist.append('.')
                strlist.append('\n')
                break
    titles.write(''.join(strlist))